In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.figure import Figure
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

class DataAnalyzerApp(tk.Tk):
    def __init__(self):
        super().__init__()
        self.title("Data Analyzer")
        self.geometry("800x600")
        self.configure(bg="#f0f0f0")  # Couleur de fond
        
        self.file_path = ""
        self.data = None
        self.numeric_cols = None
        self.selected_columns = []
        self.graph_type = tk.StringVar(value="Histogramme")
        self.color_var = tk.StringVar(value="blue")
        
        self.create_widgets()

    def create_widgets(self):
        self.controls_frame = tk.Frame(self, bg="#f0f0f0")  # Couleur de fond pour le frame de contrôles
        self.controls_frame.pack(side=tk.TOP, fill=tk.X, padx=10, pady=10)

        self.file_label = tk.Label(self.controls_frame, text="Aucun fichier sélectionné", bg="#f0f0f0", fg="blue")  # Couleur du texte
        self.file_label.pack(side=tk.LEFT)

        self.load_button = tk.Button(self.controls_frame, text="Charger fichier", command=self.load_file, bg="#007bff", fg="white")  # Couleur de fond et de texte
        self.load_button.pack(side=tk.LEFT, padx=10)

        self.columns_var = tk.StringVar()
        self.columns_dropdown = tk.OptionMenu(self.controls_frame, self.columns_var, "")
        self.columns_dropdown.pack(side=tk.LEFT, padx=10)

        self.graph_type_dropdown = tk.OptionMenu(self.controls_frame, self.graph_type, "Histogramme", "Nuage de points", "Boxplot")
        self.graph_type_dropdown.pack(side=tk.LEFT, padx=10)

        self.color_dropdown = tk.OptionMenu(self.controls_frame, self.color_var, "blue", "red", "green", "black")
        self.color_dropdown.pack(side=tk.LEFT, padx=10)

        self.stats_button = tk.Button(self.controls_frame, text="Afficher statistiques", command=self.show_statistics, bg="#28a745", fg="white")
        self.stats_button.pack(side=tk.LEFT, padx=10)

        self.plot_button = tk.Button(self.controls_frame, text="Afficher graphique", command=self.show_plot, bg="#dc3545", fg="white")
        self.plot_button.pack(side=tk.LEFT, padx=10)

        self.attributes_label = tk.Label(self.controls_frame, text="Attributs choisis : ", bg="#f0f0f0")
        self.attributes_label.pack(side=tk.LEFT, padx=10)

        self.plot_frame = tk.Frame(self, bg="#ffffff")  # Couleur de fond pour le frame de graphique
        self.plot_frame.pack(side=tk.TOP, fill=tk.BOTH, expand=True, padx=10, pady=10)

    def load_file(self):
        self.file_path = filedialog.askopenfilename(filetypes=[("CSV Files", "*.csv"), ("Text Files", "*.txt")])
        if self.file_path:
            try:
                self.data = pd.read_csv(self.file_path)
                self.numeric_cols = self.data.select_dtypes(include=[np.number]).columns
                self.selected_columns = []
                self.update_columns_dropdown()
                self.file_label.config(text=f"Fichier sélectionné : {self.file_path}")
            except Exception as e:
                messagebox.showerror("Erreur", f"Impossible de charger le fichier : {str(e)}")

    def update_columns_dropdown(self):
        menu = self.columns_dropdown["menu"]
        menu.delete(0, "end")
        for col in self.numeric_cols:
            menu.add_command(label=col, command=lambda c=col: self.select_column(c))
    
    def select_column(self, col):
        if col not in self.selected_columns:
            self.selected_columns.append(col)
            self.update_attributes_label()

    def update_attributes_label(self):
        self.attributes_label.config(text="Attributs choisis : " + ", ".join(self.selected_columns))

    def show_statistics(self):
        if self.data is not None and not self.data.empty and self.selected_columns:
            stats = self.data[self.selected_columns].describe()
            messagebox.showinfo("Statistiques", stats.to_string())
        else:
            messagebox.showinfo("Statistiques", "Aucune donnée à afficher ou aucune colonne sélectionnée")

    def show_plot(self):
        if self.data is not None and not self.data.empty:
            fig = Figure(figsize=(6, 6))
            ax = fig.add_subplot(111)
            if self.graph_type.get() == "Histogramme":
                for col in self.selected_columns:
                    ax.hist(self.data[col], bins=20, alpha=0.5, label=col, color=self.color_var.get())
                ax.set_title("Histogramme des données")
            elif self.graph_type.get() == "Nuage de points":
                for col in self.selected_columns:
                    ax.scatter(self.data.index, self.data[col], label=col, color=self.color_var.get())
                ax.set_title("Nuage de points des données")
            elif self.graph_type.get() == "Boxplot":
                ax.boxplot([self.data[col] for col in self.selected_columns], labels=self.selected_columns)
                ax.set_title("Boxplot des données")
            
            ax.legend()
            ax.set_xlabel("Valeurs")
            ax.set_ylabel("Fréquence / Valeurs")
            
            canvas = FigureCanvasTkAgg(fig, master=self.plot_frame)
            canvas.draw()
            canvas.get_tk_widget().pack(side=tk.TOP, fill=tk.BOTH, expand=True)

if __name__ == "__main__":
    app = DataAnalyzerApp()
    app.mainloop()
